## Reading OME-Zarr Data

## Learning Objectives

- Understand that OME-Zarr data is **lazily loaded**
- Understand how **compression** and **chunking** limits fetched data
- Learn how to load OME-Zarr datasets in Python

### Install and import notebook depedencies

In [1]:
import sys

!{sys.executable} -m pip install -q zarr 'fsspec[http]' ngff-zarr

In [16]:
from zarr.storage import FSStore
from ngff_zarr import from_ngff_zarr
import numpy as np
from rich import print

## Lazy Loading

In OME-Zarr, pixel data is not loaded until it is used, i.e. it is *lazily loaded*.

In [21]:
# Access an OME-Zarr via HTTPS
store = FSStore('https://s3.us-west-2.amazonaws.com/aind-open-data/SmartSPIM_631680_2022-09-09_13-52-33_stitched_2022-11-10_17-18-18/processed/OMEZarr/Ex_647_Em_690.zarr')
multiscales = from_ngff_zarr(store)
print(multiscales)

Multiscales(
    images=[
        NgffImage(
            data=dask.array<from-zarr, shape=(1, 1, 4200, 10240, 7400), dtype=uint16, chunksize=(1, 1, 1, 10240, 
7400), chunktype=numpy.ndarray>,
            dims=['t', 'c', 'z', 'y', 'x'],
            scale={'t': 1.0, 'c': 1.0, 'z': 2.0, 'y': 1.8, 'x': 1.8},
            translation={'t': 0.0, 'c': 0.0, 'z': 0.0, 'y': 0.0, 'x': 0.0},
            name='image',
            axes_units={'t': 'millisecond', 'c': None, 'z': 'micrometer', 'y': 'micrometer', 'x': 'micrometer'}
        ),
        NgffImage(
            data=dask.array<from-zarr, shape=(1, 1, 2100, 5120, 3700), dtype=uint16, chunksize=(1, 1, 1, 5120, 
3700), chunktype=numpy.ndarray>,
            dims=['t', 'c', 'z', 'y', 'x'],
            scale={'t': 1.0, 'c': 1.0, 'z': 4.0, 'y': 3.6, 'x': 3.6},
            translation={'t': 0.0, 'c': 0.0, 'z': 0.0, 'y': 0.0, 'x': 0.0},
            name='image',
            axes_units={'t': 'millisecond', 'c': None, 'z': 'micrometer', 'y': 'micrometer', 'x': 'micrometer'}
        ),
        NgffImage(
            data=dask.array<from-zarr, shape=(1, 1, 1050, 2560, 1850), dtype=uint16, chunksize=(1, 1, 1, 2560, 
1850), chunktype=numpy.ndarray>,
            dims=['t', 'c', 'z', 'y', 'x'],
            scale={'t': 1.0, 'c': 1.0, 'z': 8.0, 'y': 7.2, 'x': 7.2},
            translation={'t': 0.0, 'c': 0.0, 'z': 0.0, 'y': 0.0, 'x': 0.0},
            name='image',
            axes_units={'t': 'millisecond', 'c': None, 'z': 'micrometer', 'y': 'micrometer', 'x': 'micrometer'}
        ),
        NgffImage(
            data=dask.array<from-zarr, shape=(1, 1, 525, 1280, 925), dtype=uint16, chunksize=(1, 1, 1, 1280, 925), 
chunktype=numpy.ndarray>,
            dims=['t', 'c', 'z', 'y', 'x'],
            scale={'t': 1.0, 'c': 1.0, 'z': 16.0, 'y': 14.4, 'x': 14.4},
            translation={'t': 0.0, 'c': 0.0, 'z': 0.0, 'y': 0.0, 'x': 0.0},
            name='image',
            axes_units={'t': 'millisecond', 'c': None, 'z': 'micrometer', 'y': 'micrometer', 'x': 'micrometer'}
        ),
        NgffImage(
            data=dask.array<from-zarr, shape=(1, 1, 262, 640, 462), dtype=uint16, chunksize=(1, 1, 1, 640, 462), 
chunktype=numpy.ndarray>,
            dims=['t', 'c', 'z', 'y', 'x'],
            scale={'t': 1.0, 'c': 1.0, 'z': 32.0, 'y': 28.8, 'x': 28.8},
            translation={'t': 0.0, 'c': 0.0, 'z': 0.0, 'y': 0.0, 'x': 0.0},
            name='image',
            axes_units={'t': 'millisecond', 'c': None, 'z': 'micrometer', 'y': 'micrometer', 'x': 'micrometer'}
        )
    ],
    metadata=Metadata(
        axes=[
            Axis(name='t', type='time', unit='millisecond'),
            Axis(name='c', type='channel', unit=None),
            Axis(name='z', type='space', unit='micrometer'),
            Axis(name='y', type='space', unit='micrometer'),
            Axis(name='x', type='space', unit='micrometer')
        ],
        datasets=[
            Dataset(path='0', coordinateTransformations=[Scale(scale=[1.0, 1.0, 2.0, 1.8, 1.8], type='scale')]),
            Dataset(path='1', coordinateTransformations=[Scale(scale=[1.0, 1.0, 4.0, 3.6, 3.6], type='scale')]),
            Dataset(path='2', coordinateTransformations=[Scale(scale=[1.0, 1.0, 8.0, 7.2, 7.2], type='scale')]),
            Dataset(
                path='3',
                coordinateTransformations=[Scale(scale=[1.0, 1.0, 16.0, 14.4, 14.4], type='scale')]
            ),
            Dataset(
                path='4',
                coordinateTransformations=[Scale(scale=[1.0, 1.0, 32.0, 28.8, 28.8], type='scale')]
            )
        ],
        name='image',
        version='0.4',
        coordinateTransformations=[]
    ),
    scale_factors=None,
    method=None,
    chunks=None
)

The previous cell loaded quickly because we did not need to download all the data in the image first!

How large is pixel data for the highest resolution scale?

In [18]:
print(multiscales.images[0].data.nbytes)

636518400000

> 600 GB!

What is the structure and content of the pixel data? How do we access pixel values

[Dask](https://www.dask.org/) is a Python library that allows use to lazily load the Zarr array.

We can see the structure:

In [19]:
multiscales.images[0].data

dask.array<from-zarr, shape=(1, 1, 4200, 10240, 7400), dtype=uint16, chunksize=(1, 1, 1, 10240, 7400), chunktype=numpy.ndarray>

And load regions of the pixel data on demand.

Note that when loading the data, it is transferred in its highly compressed form. Only the chunks than are required to provide the requested region are loaded.

Use `np.asarray` with NumPy indexing on the Dask Array for an memory `numpy.ndarray`'s.

In [20]:
print(np.asarray(multiscales.images[0].data[0,0,0,:8:8]))

[[ 0 11 10 ...  3  0  0]]

## Exercises

### Exercise 1: Compare multiscale sizes

How do the sizes of the downsampled scales compare to the original resolution data?

In [33]:
# %load Reading_OME-Zarr_Data_Exercise_1_Solution.py

### Exercise 2: Fetch image

Fetch all the image pixel data for scale 4.

Print the image metadata for scale 4.

In [36]:
# %load Reading_OME-Zarr_Data_Exercise_2_Solution.py